Limb Darkening
============================

Setup
-----------------------------

Let's first make sure we have the latest version of PHOEBE 2.3 installed (uncomment this line if running in an online notebook session such as colab).

In [1]:
!pip install -I "phoebe>=2.3,<2.4"

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement phoebe<2.4,>=2.3 (from versions: 2.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.0.5, 2.0.6, 2.0.7, 2.0.8, 2.0.9, 2.0.10, 2.0.11, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.1.5, 2.1.6, 2.1.7, 2.1.8, 2.1.9, 2.1.10, 2.1.11, 2.1.12, 2.1.13, 2.1.14, 2.1.15, 2.1.16, 2.1.17, 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for phoebe<2.4,>=2.3
You should consider upgrading via the '/usr/bin/python2 -m pip install --upgrade pip' command.


As always, let's do imports and initialize a logger and a new bundle. 

In [2]:
import phoebe

logger = phoebe.logger()

b = phoebe.default_binary()

We'll just add an 'lc' dataset

In [3]:
b.add_dataset('lc', times=phoebe.linspace(0,1,101), dataset='lc01')

<ParameterSet: 73 parameters | contexts: figure, compute, dataset, constraint>

Relevant Parameters
-----------------------------

`ld_mode_bol`, `ld_func_bol`, `ld_coeffs_source_bol`, and `ld_coeffs_bol` are for bolometric limb-darkening, which is only currently used for [irradiation/reflection](./reflection_heating.ipynb).  These are per-component parameters that have context='component'.

In [4]:
print(b['ld_mode_bol@primary'])

Parameter: ld_mode_bol@primary@component
                       Qualifier: ld_mode_bol
                     Description: Mode to use for bolometric limb-darkening (used only for irradiation).
                           Value: lookup
                         Choices: lookup, manual
                  Constrained by: 
                      Constrains: None
                      Related to: None



In [5]:
print(b['ld_func_bol@primary'])

Parameter: ld_func_bol@primary@component
                       Qualifier: ld_func_bol
                     Description: Bolometric limb darkening model (used only for irradiation).
                           Value: logarithmic
                         Choices: linear, logarithmic, quadratic, square_root, power
                  Constrained by: 
                      Constrains: None
                      Related to: None



In [6]:
print(b['ld_func_bol@primary'].choices)

['linear', 'logarithmic', 'quadratic', 'square_root', 'power']


### ld_mode_bol = 'lookup'

By default, `ld_mode_bol` is set to 'lookup', in which case the coefficients are pulled from the atmosphere table (according to `ld_coeffs_source_bol`) per-component (but not per-element).  Note that this differs slightly from the dataset-treatment where the lookup is handled per-element.

In [7]:
print(b['ld_coeffs_source_bol@primary'])

Parameter: ld_coeffs_source_bol@primary@component
                       Qualifier: ld_coeffs_source_bol
                     Description: Source for bolometric limb darkening coefficients (used only for irradiation; 'auto' to interpolate from the applicable table according to the 'atm' parameter, or the name of a specific atmosphere table)
                           Value: auto
                         Choices: auto, ck2004, phoenix
                  Constrained by: 
                      Constrains: None
                      Related to: None
                 Only visible if: ld_mode_bol:lookup



To access the interpolated values that will be used under-the-hood, we can call [b.compute_ld_coeffs](../api/phoebe.frontend.bundle.Bundle.compute_ld_coeffs.md).

(To only compute the bolometric quantities, pass `dataset='bol'`)

In [8]:
b.compute_ld_coeffs(dataset='bol')

{'ld_coeffs_bol@primary@component': array([0.70974057, 0.2862986 ]),
 'ld_coeffs_bol@secondary@component': array([0.70974057, 0.2862986 ])}

### ld_mode_bol = 'manual'

To pass coefficients manually, we can change `ld_mode_bol` to 'manual'

In [9]:
b.set_value_all('ld_mode_bol', value='manual')

In [10]:
print(b['ld_coeffs_bol@primary'])

Parameter: ld_coeffs_bol@primary@component
                       Qualifier: ld_coeffs_bol
                     Description: Bolometric limb darkening coefficients (used only for irradiation).
                           Value: [0.5 0.5]
                  Constrained by: 
                      Constrains: None
                      Related to: None
                 Only visible if: ld_mode_bol:manual



All other limb-darkening parameters (`ld_mode`, `ld_func`, `ld_coeffs_source`, and `ld_coeffs`) are per-component and per-dataset parameters with context='dataset'.

In [11]:
print(b.filter(qualifier='ld_mode', dataset='lc01'))

ParameterSet: 2 parameters
     ld_mode@primary@lc01@dataset: interp
   ld_mode@secondary@lc01@dataset: interp


### ld_mode = 'interp'

Unlike bolometric limb-darkening, passband limb-darkening coefficients can be interpolated directly from atmosphere tables, this is the default case, with `ld_mode` set to 'interp'.  

Note that before PHOEBE 2.2, this was accomplished by setting `ld_func` to 'interp' (see [this explanation for migrating from earlier versions](./21_22_ld_coeffs_source.ipynb)).

In [12]:
print(b.get_parameter(qualifier='ld_mode', dataset='lc01', component='primary').choices)

['interp', 'lookup', 'manual']


Note that `ld_func`, `ld_coeffs_source`, and `ld_coeffs` aren't visible (relevant) if `ld_mode` is set to  'interp' (which it is by default).

In [13]:
print(b.filter(qualifier='ld*', dataset='lc01'))

ParameterSet: 2 parameters
     ld_mode@primary@lc01@dataset: interp
   ld_mode@secondary@lc01@dataset: interp


### ld_mode = 'lookup'

Setting the value of `ld_mode` to 'lookup' will expose the `ld_func` and `ld_coeffs_source` parameters.  Note that this behavior is slightly new as of PHOEBE 2.2 (see [this explanation for migrating from earlier versions](./21_22_ld_coeffs_source.ipynb)).

When set to 'lookup', then the limb-darkening will be interpolated **per-element** (for PHOEBE 2, other backends may interpolate per-star) for the function given in `ld_func` from the atmosphere table dictated by the `ld_coeffs_source` parameter (or the `atm` parameter and falling back on 'ck2004' if no match is found if `ld_coeffs_source` is set to 'auto').

Note that the bolometric `ld_mode_bol` of 'lookup' interpolates per-component, but not per-element.

In [14]:
b.set_value(qualifier='ld_mode', dataset='lc01', component='primary', value='lookup')

In [15]:
print(b.filter(qualifier='ld*', dataset='lc01', component='primary'))

ParameterSet: 3 parameters
     ld_mode@primary@lc01@dataset: lookup
     ld_func@primary@lc01@dataset: logarithmic
   ld_coeffs_source@primary@lc...: auto


To manually choose an available atmosphere table, you can choose some other value ('ck2004', for example).

In [16]:
print(b.get_parameter(qualifier='ld_coeffs_source', dataset='lc01', component='primary'))

Parameter: ld_coeffs_source@primary@lc01@dataset
                       Qualifier: ld_coeffs_source
                     Description: Source for limb darkening coefficients ('auto' to interpolate from the applicable table according to the 'atm' parameter, or the name of a specific atmosphere table)
                           Value: auto
                         Choices: auto, phoenix, ck2004
                  Constrained by: 
                      Constrains: None
                      Related to: None
                 Only visible if: ld_mode:lookup



Although not necesary, we can access the interpolated coefficients by calling [b.compute_ld_coeffs](../api/phoebe.frontend.bundle.Bundle.compute_ld_coeffs.md).

In [17]:
print(b.compute_ld_coeffs(dataset='lc01'))

{'ld_coeffs@primary@lc01': array([0.78822538, 0.27905826])}


### ld_mode = 'manual'

Setting the value of `ld_mode` to 'manual' will expose the `ld_func` and `ld_coeffs` parameters.  Again, note that this behavior is slightly new as of PHOEBE 2.2 (see [this explanation for migrating from earlier versions](./21_22_ld_coeffs_source.ipynb)).

In this case, we can manually provide the coefficients through the `ld_coeffs` parameter, keeping care that they are of the correct length for the given value of `ld_func`.  To ensure this is the case, call [b.run_checks](../api/phoebe.frontend.bundle.Bundle.run_checks.md) (or wait until [b.run_compute](../api/phoebe.frontend.bundle.Bundle.run_compute.md) which will raise an error if the length is in conflict).

In [18]:
b.set_value(qualifier='ld_mode', dataset='lc01', component='primary', value='manual')

In [19]:
print(b.filter(qualifier='ld*', dataset='lc01', component='primary'))

ParameterSet: 3 parameters
     ld_mode@primary@lc01@dataset: manual
     ld_func@primary@lc01@dataset: logarithmic
   ld_coeffs@primary@lc01@dataset: [0.5 0.5]


In [20]:
print(b.get_parameter(qualifier='ld_coeffs', dataset='lc01', component='primary'))

Parameter: ld_coeffs@primary@lc01@dataset
                       Qualifier: ld_coeffs
                     Description: Limb darkening coefficients
                           Value: [0.5 0.5]
                  Constrained by: 
                      Constrains: None
                      Related to: None
                 Only visible if: ld_mode:manual

